In [1]:
from pathlib import Path
import csv
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Input
from keras import models

/Users/junweiluo/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/junweiluo/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/junweiluo/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/junweiluo/anaconda3/lib/python3.7/site-pac

In [2]:
csvpath = Path('database.csv')

In [3]:
df = pd.read_csv(csvpath)

In [4]:
df = df.iloc[:149,:30]

In [5]:
df

Player      T1_1      T1_2      T2_1      T2_2      T3_1  \
0                Ty_Cobb  0.949931  0.122283  0.900549  0.374457  0.664609   
1              Babe_Ruth  0.631318  0.159712  0.805748  0.376259  0.677899   
2             Lou_Gehrig  0.613402  0.046107  0.787371  0.425205  0.574742   
3             Tony_Gwynn  0.519866  0.141333  0.671135  0.391333  0.506462   
4             Al_Simmons  0.353535  0.062745  0.270202  0.447059  0.505051   
..                   ...       ...       ...       ...       ...       ...   
144  marquis_grissom_953  0.291379  0.045768  0.261207  0.391240  0.528448   
145    ivan_calderon_952  0.359259  0.046036  0.173148  0.223785  0.403704   
146   wally_westlake_951  0.292328  0.081132  0.072751  0.387736  0.498677   
147  bobby_higginson_950  0.645186  0.032242  0.906832  0.410218  0.670807   
148     marcel_ozuna-949  0.317308  0.075843  0.180556  0.452949  0.429487   

         T3_2      H1_1      H1_2      H2_1  ...   RBI   SB    OBP    SLG  \
0    0.321739  0.679010  0.018480  0.698220  ...  1944  897  0.433  0.512   
1    0.395683  0.628345  0.233813  0.682854  ...  2214  123  0.474  0.690   
2    0.448770  0.519330  0.218240  0.640460  ...  1995  102  0.447  0.632   
3    0.480667  0.520820  0.282670  0.617520  ...  1138  319  0.388  0.459   
4    0.460784  0.388890  0.133330  0.419190  ...  1828   88  0.380  0.535   
..        ...       ...       ...       ...  ...   ...  ...    ...    ...   
144  0.477854  0.442240  0.242620  0.368100  ...   967  429  0.318  0.415   
145  0.337596  0.400000  0.075450  0.401850  ...   444   97  0.333  0.442   
146  0.497170  0.421960  0.174530  0.464290  ...   539   19  0.345  0.450   
147  0.473710  0.622670  0.230660  0.680900  ...   709   91  0.358  0.455   
148  0.525281  0.442310  0.262640  0.344020  ...   591  115  0.332  0.386   

       OPS  OPS+      BA  Top  Middle  Bottom  
0    0.944   168  0.3660    1       0       0  
1    1.164   206  0.3420    1       0       0  
2    1.080   179  0.3400    1       0       0  
3    0.847   132  0.3380    1       0       0  
4    0.915   133  0.3340    1       0       0  
..     ...   ...     ...  ...     ...     ...  
144  0.732    92  0.2720    0       0       1  
145  0.775   113  0.2720    0       0       1  
146  0.795   111  0.2721    0       0       1  
147  0.813   113  0.2721    0       0       1  
148  0.718    93  0.2723    0       0       1  

[149 rows x 30 columns]

In [6]:
def getTier(Top, Middle, Bottom):
    if Top ==1:
        Tier = 0
    elif Middle==1:
        Tier = 1
    else:
        Tier = 2
    return Tier


df['tier'] = df.apply(lambda x: getTier(x['Top'], x['Middle'], x['Bottom']),axis=1)

In [7]:
def getAngle(a, b, c, d):
    ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    if ang < 0:
        ang += 360
    if d == 1:
        ang = 360 - ang
    return ang

df['Angle1'] = df.apply(lambda x: getAngle((x['T1_1'],x['T1_2']),(x['T2_1'],x['T2_2']),(x['T3_1'],x['T3_2']),x['hand']),axis=1)
df['Angle2'] = df.apply(lambda x: getAngle((x['H1_1'],x['H1_2']),(x['H2_1'],x['H2_2']),(x['H3_1'],x['H3_2']),x['hand']),axis=1)

In [8]:
#new_df = df[['Angle1', 'Angle2','hand', 'height','weight','AB','H','HR','R','RBI','SB','OBP','SLG','OPS','OPS+']]
new_df = df[['Angle1', 'Angle2','hand', 'height','weight','HR','R','RBI','SB','OBP','SLG','OPS','OPS+']]

In [9]:
X= new_df.to_numpy()

In [10]:
y = df[['BA']].to_numpy()

In [11]:
Y = df[['tier']].to_numpy()

In [12]:
scaler = StandardScaler()
X_transformed = scaler.fit_transform(X)

In [13]:
X_train, X_valid, y_train, y_valid, Y_train, Y_valid = train_test_split(X_transformed, 
                                                    y, Y,
                                                    random_state=1)

In [14]:
tf.random.set_seed(41)
np.random.seed(41)

input = keras.layers.Input(shape=[13], name="input")
hidden1 = keras.layers.Dense(30, activation="relu")(input)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
output = keras.layers.Dense(3, activation='softmax',name="output")(hidden2)
aux_output = keras.layers.Dense(1, name='aux_output')(hidden2)
model = keras.models.Model(inputs=[input], outputs=[output, aux_output])

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 13)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           420         input[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           930         dense[0][0]                      
__________________________________________________________________________________________________
output (Dense)                  (None, 3)            93          dense_1[0][0]                    
______________________________________________________________________________________________

In [16]:
model.compile(loss = ['sparse_categorical_crossentropy','mean_squared_error'], 
              optimizer='adam', loss_weights=[0.5,0.5], metrics=['accuracy'])

In [17]:
history = model.fit(X_train, (Y_train,y_train), epochs=50)

Train on 111 samples
Epoch 1/50
111/111 [==============================] - 0s 4ms/sample - loss: 0.7837 - output_loss: 1.1246 - aux_output_loss: 0.4341 - output_accuracy: 0.3514 - aux_output_accuracy: 0.0000e+00
Epoch 2/50
111/111 [==============================] - 0s 109us/sample - loss: 0.6815 - output_loss: 1.0953 - aux_output_loss: 0.2543 - output_accuracy: 0.3964 - aux_output_accuracy: 0.0000e+00
Epoch 3/50
111/111 [==============================] - 0s 123us/sample - loss: 0.6239 - output_loss: 1.0923 - aux_output_loss: 0.1531 - output_accuracy: 0.4505 - aux_output_accuracy: 0.0000e+00
Epoch 4/50
111/111 [==============================] - 0s 119us/sample - loss: 0.5889 - output_loss: 1.0725 - aux_output_loss: 0.0930 - output_accuracy: 0.4505 - aux_output_accuracy: 0.0000e+00
Epoch 5/50
111/111 [==============================] - 0s 154us/sample - loss: 0.5688 - output_loss: 1.0815 - aux_output_loss: 0.0747 - output_accuracy: 0.4955 - aux_output_accuracy: 0.0000e+00
Epoch 6/50
111/1

In [18]:
total_loss = model.evaluate(X_valid, [Y_valid, y_valid])

38/38 [==============================] - 0s 3ms/sample - loss: 0.4174 - output_loss: 0.8931 - aux_output_loss: 0.0231 - output_accuracy: 0.6579 - aux_output_accuracy: 0.0000e+00
